In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = 'https://worldpopulationreview.com/countries'
page=requests.get(url)
soup= BeautifulSoup(page.text, 'html')
print(soup)

<!DOCTYPE html>
<html class="__variable_ba1713 __variable_d65c78" lang="en"><head><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1" name="viewport"/><link as="font" crossorigin="" href="/_next/static/media/a34f9d1faa5f3315-s.p.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="" href="/_next/static/media/f8a669a999c483ca-s.p.woff2" rel="preload" type="font/woff2"/><link as="image" fetchpriority="high" imagesrcset="/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Flogo2x.b874b885.png&amp;w=32&amp;q=75 1x, /_next/image?url=%2F_next%2Fstatic%2Fmedia%2Flogo2x.b874b885.png&amp;w=64&amp;q=75 2x" rel="preload"/><link as="image" fetchpriority="high" imagesrcset="/_next/image?url=%2Fsvg_placeholder.png&amp;w=1080&amp;q=75 1x, /_next/image?url=%2Fsvg_placeholder.png&amp;w=2048&amp;q=75 2x" rel="preload"/><link as="image" href="https://s3.amazonaws.com/images.wpr.com/flag-pages/flags-iso/shiny/64/IN.png" rel="preload"/><link as="image" href="https://s3.amazon

In [2]:
table=soup.find_all('table')[0]
columns=table.find_all('th')
print(columns)

[<th class="datatable-th sticky top-0 border-none border-wpr-table_border bg-wpr-table_header_bg px-3 py-2 text-left align-bottom leading-4 text-wpr-table_header md:px-4 pin left-0 z-50"><div class="relative flex flex-col items-stretch justify-start bg-wpr-table_header_bg"><div class="flex flex-row items-center cursor-pointer select-none">Flag</div></div></th>, <th class="datatable-th sticky top-0 border-none border-wpr-table_border bg-wpr-table_header_bg px-3 py-2 text-left align-bottom leading-4 text-wpr-table_header md:px-4 z-40"><div class="relative flex flex-col items-stretch justify-start bg-wpr-table_header_bg"><div class="flex flex-row items-center cursor-pointer select-none">Country</div></div></th>, <th class="datatable-th sticky top-0 border-none border-wpr-table_border bg-wpr-table_header_bg px-3 py-2 text-left align-bottom leading-4 text-wpr-table_header md:px-4 z-40"><div class="relative flex flex-col items-stretch justify-start bg-wpr-table_header_bg"><div class="flex fl

In [3]:
title_data = [data.text.strip() for data in columns]
column_data = pd.DataFrame(columns=title_data)
column_data = pd.DataFrame(columns=title_data[1:11])
print(column_data)

Empty DataFrame
Columns: [Country, 2024 Population, 2023 Population, Area (km²), Land Area (km²), Density (/km²), Change, World %, Rank, UN Member]
Index: []


In [ ]:
row_data = table.find_all('tr')

for row in row_data[1:]:
    table_data = row.find_all('td')
    all_rows = []
    for i, cell in enumerate(table_data):
        column_name = column_data.columns[i]
        if column_name == "UN Member":
            svg = cell.find("svg")
            if svg and "tabler-icon-check" in svg.get("class", []):
                all_rows.append(True)
            elif svg and "tabler-icon-x" in svg.get("class", []):
                all_rows.append(False)
            else:
                all_rows.append(None)
        else:
            all_rows.append(cell.text.strip())
    if len(all_rows) == len(column_data.columns):
        length = len(column_data)
        column_data.loc[length] = all_rows
    else: 
        print(f"Skipping row due to mismatched columns. Expected {len(column_data.columns)} columns, got {len(all_rows)}.")

In [ ]:
import pandas as pd

display(column_data)
column_data.to_csv("population-dataset.csv", index=False)
print("Data saved to column_data.csv")

,Country,2024 Population,2023 Population,Area (km²),Land Area (km²),Density (/km²),Change,World %,Rank,UN Member
0,India,"1,450,940,000","1,438,070,000",3.3M,3M,488,0.89%,18.13%,1,True
1,China,"1,419,320,000","1,422,580,000",9.7M,9.4M,151,-0.23%,17.73%,2,True
2,United States,"345,427,000","343,477,000",9.4M,9.1M,38,0.57%,4.32%,3,True
3,Indonesia,"283,488,000","281,190,000",1.9M,1.9M,151,0.82%,3.54%,4,True
4,Pakistan,"251,269,000","247,504,000",881.9K,770.9K,326,1.52%,3.14%,5,True
...,...,...,...,...,...,...,...,...,...,...
229,Montserrat,"4,389","4,420",102,102,43,-0.7%,,230,False
230,Falkland Islands,"3,470","3,477",12.2K,12.2K,0.29,-0.2%,,231,False
231,Tokelau,"2,506","2,397",12,10,251,4.55%,,232,False
232,Niue,"1,819","1,817",261,261,7,0.11%,,233,False


Data saved to column_data.csv


In [15]:
import json

countries = column_data.iloc[:, 0].tolist()

with open("countries.json", "w") as file:
    json.dump(countries, file, indent=4)

print("Data saved to countries.json")

Data saved to countries.json


In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://worldpopulationreview.com/countries/india"

headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    tables = soup.find_all("table")

    for index, table in enumerate(tables):
        df = pd.read_html(str(table))[0]
        print(f"Table {index+1}:")
        print(df)
        print("\n" + "-"*50 + "\n")

else:
    print("Failed to retrieve the webpage. Status code:", response.status_code)

Table 1:
                                     0           1
0  India Population (as of 11/19/2024)  1455993730
1      Last UN Estimate (July 1, 2024)  1450940000
2                       Births per Day       63483
3                       Deaths per Day       26166
4                   Migrations per Day       -1727
5                   Net Change per Day       35590
6       Population Change Since Jan. 1    11531160

--------------------------------------------------

Table 2:
                                        0   1
0               One birth every 1 seconds NaN
1               One death every 3 seconds NaN
2           One emigrant every 50 seconds NaN
3  Net gain of one person every 2 seconds NaN

--------------------------------------------------

Table 3:
        City  2024 Pop.
0     Mumbai   12691836
1      Delhi   10927986
2  Bengaluru    5104047
3    Kolkata    4631392
4    Chennai    4328063
5  Ahmedabad    3719710
6  Hyderabad    3597816
7       Pune    2935744
8      Surat 

/var/folders/33/tmt_dn957d99k_t2zvbqlw_40000gn/T/ipykernel_36650/294880041.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/var/folders/33/tmt_dn957d99k_t2zvbqlw_40000gn/T/ipykernel_36650/294880041.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/var/folders/33/tmt_dn957d99k_t2zvbqlw_40000gn/T/ipykernel_36650/294880041.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
/var/folders/33/tmt_dn957d99k_t2zvbqlw_40000gn/T/ipykernel_36650/294880041.py:15: FutureWarning: Passing literal html to 'read_html' is deprec